In [ ]:
# environment setup for server

# install tools
!pip install jedi huggingface_hub pyngrok nest_asyncio fastapi uvicorn -q

# install precompiled llama-cpp-python with server support
!pip install llama-cpp-python[server] --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu122 -q

import nest_asyncio
from huggingface_hub import hf_hub_download
from pyngrok import ngrok
import threading
import subprocess

nest_asyncio.apply()

In [ ]:
# download fine-tuned model from Hugging Face Hub
REPO_ID = "tspec2/redstring-8gb"
FILENAME = "meta-llama-3.1-8b.Q8_0.gguf" 

print(f"Fetching {FILENAME} from {REPO_ID}...")

model_path = hf_hub_download(
    repo_id=REPO_ID,
    filename=FILENAME
)

print(f"Model cached at: {model_path}")

In [ ]:
# start the server
NGROK_TOKEN = "39pFdrEYO2brshKa669babIg7Nr_5pY9xXUPgS7rumDHdeu5Y"

!ngrok config add-authtoken {NGROK_TOKEN}
ngrok.kill()
public_url = ngrok.connect(8000).public_url

def start_api():
    cmd = f"python3 -m llama_cpp.server --model {model_path} --n_gpu_layers -1 --n_ctx 8192 --host 0.0.0.0 --port 8000"
    subprocess.run(cmd, shell=True)

threading.Thread(target=start_api, daemon=True).start()
print(f"\n✨ API IS LIVE! ✨")
print(f"Copy this into your local app: {public_url}/v1")